## 데이트팝 매장 크롤링 후, 음식점과 놀이매장으로 분리

### 기존 데이트팝 컨텐츠 매장 크롤링
- 데이트팝 컨텐츠 활성화 매장을 대상으로, 네이버지도와 인스타그램의 정보를 크롤링합니다.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from urllib.parse import urlparse, parse_qs
from shapely.geometry import Point
import time
import re

from utils.load_hotspots import load_hotspots
from utils.haversine import haversine
from utils.convert_str_to_number import convert_str_to_number

In [3]:
columns = ['store_id', 'name', 'category', 'is_food', 'new_store', 'instagram_link', 'instagram_post', 'instagram_follower', 'hot_spot',
           'visitor_review_count', 'blog_review_count', 'distance_from_subway', 'on_tv', 'parking_available', 'no_kids',
           'seoul_michelin', 'age-2030', 'gender-balance', 'on_blue_ribbon', 'image_urls', 'address', 'phone', 'gps']

In [6]:
class DatePopCrawler:
    def __init__(self, hotspots, blue_ribbon, datepop_stores):

        self.hotspots = hotspots
        self.blue_ribbon = blue_ribbon
        self.datapop_stores = datepop_stores
        self.current_store = None  # 현재 찾으려는 매장 정보 저장

        # 네이버지도에서 현재 보고 있는 매장의 정보 저장
        self.store_dict = None

        # 크롤링 결과 저장
        self.data_clear = pd.DataFrame(columns=columns)

        # Iframe 전환용 xpath
        self.search_iframe = """//*[@id="searchIframe"]"""
        self.entry_iframe = """//*[@id="entryIframe"]"""
        self.empty_searchIframe = """//*[@id="_pcmap_list_scroll_container"]"""
        self.empty_entryIframe = """//*[@id="app-root"]"""
        self.empty_root = """//*[@id="root"]"""

        # Chrome driver 세팅
        self.driver = self.init_driver()
        self.wait = WebDriverWait(self.driver, 10)
        self.wait_short = WebDriverWait(self.driver, 2)

    # 크롬 드라이버 설정
    def init_driver(self):
        options = webdriver.ChromeOptions()

        options.add_argument("--enable-logging")
        options.add_argument("--v=1")  # 로그 레벨 설정

        driver = webdriver.Chrome(options=options)
        driver.get("https://map.naver.com/")

        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[0])

        return driver

    # 검색어를 input field에 입력 후 클릭
    def search_keyword(self, store_name):
        time.sleep(1)
        self.move_to_default_content()

        try:
            css_selector = ".search_wrap .search_box .btn_clear"
            clear_button = self.driver.find_element(
                By.CSS_SELECTOR, css_selector)
            clear_button.click()
        except NoSuchElementException:
            pass

        css_selector = ".input_search"
        elem = self.wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, css_selector)))

        elem.send_keys(store_name)
        time.sleep(1)
        elem.send_keys(Keys.RETURN)

    # 한 매장의 크롤링 정보를 저장하는 변수 초기화
    def init_dictionary(self):
        self.store_dict = {
            "store_id": None,
            "name": "",
            "category": "",
            "is_food": True,
            "new_store": False,
            "instagram_link": None,
            "instagram_post": None,
            "instagram_follower": None,
            "hot_spot": False,
            "visitor_review_count": 0,
            "blog_review_count": 0,
            "distance_from_subway": None,
            "on_tv": False,
            "parking_available": False,
            "no_kids": False,
            "seoul_michelin": False,
            "age-2030": None,
            "gender-balance": None,
            "on_blue_ribbon": None,
            "image_urls": [],
            "address": None,
            "phone": None,
            "gps": {
                "latitude": None,
                "longitude": None,
            }
        }

    # 한 매장에 대한 페이지 열기
    def get_into_store(self, index):
        try:
            # 매장의 정보를 저장할 저장할 dictionay 변수 초기화
            self.init_dictionary()
            # 매장 목록이 있는 search Iframe으로 이동
            self.move_to_search_iframe()

            # 현재 페이지의 index번째 매장 클릭
            store_xpath = f"""//*[@id="_pcmap_list_scroll_container"]/ul/li[{
                index}]//a[.//div[contains(@class, 'place_bluelink')]]"""
            elem = self.wait.until(EC.element_to_be_clickable((By.XPATH, store_xpath)))
            time.sleep(2)
            self.driver.execute_script("arguments[0].scrollIntoView(true);", elem)
            self.driver.execute_script("arguments[0].click()", elem)

            # entry Iframe에 접근하기위해 상위 frame으로 이동
            time.sleep(1)
            self.driver.switch_to.default_content()
            self.wait.until(EC.presence_of_element_located((By.XPATH, self.empty_root)))
            self.driver.find_element(By.XPATH, self.empty_root)

            # 매장 정보가 있는 entry Iframe으로 이동
            self.wait.until(EC.presence_of_element_located((By.XPATH, self.entry_iframe)))
            iframe_element = self.driver.find_element(By.XPATH, self.entry_iframe)
            
            # entry Iframe으로 실제로 이동하기 전에 해당 매장에 대한 url 얻기
            iframe_src = iframe_element.get_attribute('src')
            # 매장의 위도-경도 / 고유 ID 얻기
            parsed_url = urlparse(iframe_src)
            query_params = parse_qs(parsed_url.query)
            latitude = float(query_params.get('y')[0])
            longitude = float(query_params.get('x')[0])
            # store_point = Point(longitude, latitude)
            self.store_dict["gps"] = {
                "latitude": latitude,
                "longitude": longitude
            }
            path_segments = parsed_url.path.split('/')
            store_id = path_segments[2]
            self.store_dict["store_id"] = store_id

            if self.check_same_store() == False:
                return False


            # entry Iframe으로 이동
            self.driver.switch_to.frame(iframe_element)
            self.driver.find_element(By.XPATH, self.empty_entryIframe)

            # "요청하신 페이지를 찾을 수 없습니다" -> "새로고침" 버튼 클릭하여 매장 정보 다시 불러오기
            try:
                reset_elem = self.wait_short.until(
                    EC.presence_of_element_located((By.XPATH, """//a[contains(text(), '새로고침')]""")))

                self.driver.execute_script("arguments[0].click()", reset_elem)
                print("새로고침 발생")
            except (NoSuchElementException, TimeoutException):  # 매장 정보가 잘 불러와진 경우
                pass

            return True
        except Exception as e:
            print(e)
            return False

    # 한 매장에 대한 정보 얻기
    # Chrome Driver가 Entry Iframe에 이미 들어와있다 가정하고 진행
    def get_store_details(self):

        # 매장이 핫스팟에 위치해있는지 확인
        store_point = Point(float(self.store_dict["gps"]["longitude"]), float(
            self.store_dict["gps"]["latitude"]))
        for i in range(len(self.hotspots)):
            polygon = self.hotspots[i]["polygon_area"]
            if polygon.contains(store_point):
                self.store_dict["hot_spot"] = True  # default = False
                break

        # 매장 정보 로딩을 위한 명시적 대기
        time.sleep(3)

        # 매장 이름, 카테고리
        try:
            store_name_xpath = """//*[@id="_title"]/div/span"""
            elem = self.wait.until(EC.presence_of_all_elements_located(
                (By.XPATH, store_name_xpath)))

            self.store_dict['name'] = elem[0].text
            self.store_dict['category'] = elem[1].text
        # 드물게 매장 이름을 크롤링하지 못하는 에러 발생
        # 해당 매장 생략하고 다음 매장 크롤링 진행
        except TimeoutException as e:
            print("매장 이름, 카테고리 에러: ", e)
            return False
        except Exception as e:
            print("매장 이름, 카테고리 에러: ", e)
            return False

        # 신규 매장 여부
        # 일반 크롤링일 때만 실행
        if self.store_dict["new_store"] == False:
            try:
                new_open_xpath = """//*[@id='_title']/div/span[contains(text(), '새로오픈')]"""
                new_open_spans = self.driver.find_element(
                    By.XPATH, new_open_xpath)

                if new_open_spans:
                    self.store_dict["new_store"] = True
                else:
                    pass
            except NoSuchElementException:
                pass

        # 방문자 리뷰, 블로그 리뷰 개수
        try:
            elem_visitor = self.driver.find_element(
                By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
            visitor_review_count = int(re.findall(
                r'\d+', elem_visitor.text.replace(",", ""))[0])
            self.store_dict['visitor_review_count'] = visitor_review_count
        except NoSuchElementException:
            self.store_dict['visitor_review_count'] = None
        try:
            elem_blog = self.driver.find_element(
                By.XPATH, value="//a[contains(text(), '블로그리뷰')]")
            blog_review_count = int(re.findall(
                r'\d+', elem_blog.text.replace(",", ""))[0])
            self.store_dict['blog_review_count'] = blog_review_count
        except NoSuchElementException:
            self.store_dict['blog_review_count'] = None

        # 도로명주소
        try:
            address_xpath = "//strong[contains(.,'주소')]/following-sibling::div/a/span"
            address_elem = self.driver.find_element(By.XPATH, address_xpath)
            address_text = address_elem.text
            if address_text != "":
                self.store_dict["address"] = address_text
        except NoSuchElementException:
            self.store_dict["address"] = None
        except Exception as e:
            print("도로명주소 에러: ", e)
            self.store_dict["address"] = False

        # 매장 전화번호
        try:
            phone_xpath = "//strong[contains(.,'전화번호')]/following-sibling::div/span"
            phone_elem = self.driver.find_element(By.XPATH, phone_xpath)
            phone_text = phone_elem.text
            if phone_text != "":
                self.store_dict["phone"] = phone_text
        except NoSuchElementException:
            self.store_dict["phone"] = None
        except Exception as e:
            print("전화번호 에러: ", e)
            self.store_dict["phone"] = False

        # 인스타그램 계정 존재 확인
        try:
            elem = self.driver.find_element(
                By.XPATH, value="//a[contains(@href, 'instagram.com')]")
            instagram_url = elem.get_attribute('href')

            wrong_url = ["https://instagram.com", "https://instagram.com/", "http://instagram.com",
                         "http://instagram.com/", "instagram.com", "instagram.com/",
                         "https://www.instagram.com/", "http://www.instagram.com/", "https://www.instagram.com",
                         "http://www.instagram.com"]
            if instagram_url in wrong_url:
                self.store_dict['instagram_link'] = None
            else:
                # query parameter 제거
                if "?" in instagram_url:
                    instagram_url = instagram_url.split("?")[0]
                # 계정 이름 이후 "/" 뒷부분 모두 제거
                if instagram_url.count("/") >= 4:
                    instagram_url = "/".join(instagram_url.split("/", 4)[:4])

                self.store_dict['instagram_link'] = instagram_url
        except (NoSuchElementException, TimeoutException):
            self.store_dict['instagram_link'] = None
            self.store_dict['instagram_post'] = None
            self.store_dict['instagram_follower'] = None
        except Exception as e:
            print("인스타그램 에러:", e)
            self.store_dict['instagram_link'] = None
            self.store_dict['instagram_post'] = None
            self.store_dict['instagram_follower'] = None

        # 서울 미쉐린 가이드 등재 여부
        try:
            michelin_xpath = """//div[a[contains(text(), '미쉐린 가이드 서울')]]"""
            self.driver.find_element(By.XPATH, michelin_xpath)
            self.store_dict['seoul_michelin'] = True
        except NoSuchElementException:
            self.store_dict['seoul_michelin'] = False
        except Exception as e:
            print("서울 미쉐린 가이드 에러:", e)
            self.store_dict['seoul_michelin'] = None

        # 지하철역 출구로부터 거리
        try:
            subway_xpath = "/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div"
            elem = self.driver.find_element(By.XPATH, subway_xpath)
            text = elem.text

            numbers = re.findall(r'\d+', text)
            if numbers:
                self.store_dict["distance_from_subway"] = convert_str_to_number(numbers[-1])
        except NoSuchElementException:
            self.store_dict["distance_from_subway"] = None
        except Exception as e:
            print("지하철역 에러: ", e)
            self.store_dict["distance_from_subway"] = None

        # 방송 출연 여부
        try:
            tv_xpath = """//strong[descendant::span[text()='TV방송정보']]"""
            self.driver.find_element(By.XPATH, tv_xpath)
            self.store_dict['on_tv'] = True
        except NoSuchElementException:
            self.store_dict['on_tv'] = False
        except Exception as e:
            print("방송 출연 에러: ", e)
            self.store_dict['on_tv'] = None

        # 주차 가능, 반려동물 동반, 노키즈존
        try:
            convenient_xpath = "//strong[descendant::span[text()='편의']]/ancestor::div[1]/div/div"
            elem = self.driver.find_element(By.XPATH, convenient_xpath)
            convenients = elem.text

            for parking in ["주차", "발렛파킹"]:
                if parking in convenients:
                    self.store_dict["parking_available"] = True
                    break

            if "노키즈존" in convenients:
                self.store_dict["no_kids"] = True
        except NoSuchElementException:
            self.store_dict["parking_available"] = False
            self.store_dict["no_kids"] = False
        except Exception as e:
            print("주차, 반려동물, 노키즈 에러: ", e)
            self.store_dict["parking_available"] = False
            self.store_dict["no_kids"] = False

        # DataLab: 연령별 / 성별 검색 인기도
        try:
            # entryIframe 스크롤 끝까지 내려서 모든 컨텐츠 로딩
            last_height = self.driver.execute_script(
                "return document.body.scrollHeight")
            while True:
                self.driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(0.5)
                new_height = self.driver.execute_script(
                    "return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # DataLab 항목 찾고 해당 element로 스크롤 이동
            datalab_xpath = """//div[h2/span[contains(text(), '데이터랩')]]"""
            datalab_elem = self.driver.find_element(By.XPATH, datalab_xpath)
            self.driver.execute_script(
                "arguments[0].scrollIntoView(true);", datalab_elem)

            # "테마키워드"라는 text가 있는 경우, "더보기 버튼을 눌러줘야 연령별/성별 검색어 비율 확인 가능
            try:
                theme_keyword_xpath = """.//div/div/div/h3[contains(text(), '테마키워드')]"""
                datalab_elem.find_element(By.XPATH, theme_keyword_xpath)
                button_elem = datalab_elem.find_element(
                    By.XPATH, ".//div[2]/div/a")
                self.driver.execute_script("arguments[0].click()", button_elem)
            except NoSuchElementException:
                pass
            except Exception as e:
                print("데이터랩 더보기 버튼 에러: ", e)

            # 20대와 30대가 top 1, 2를 차지하는지 확인
            age_xpath = """//*[@id="bar_chart_container"]/ul/li/div[1]/span/span[1]"""
            age_elements = self.wait.until(
                EC.presence_of_all_elements_located((By.XPATH, age_xpath)))
            percentage_by_age = [
                round(float(item.text.replace('%', '')), 2) for item in age_elements]
            top_two = sorted(percentage_by_age, reverse=True)[:2]
            is_2030_in_top_two = percentage_by_age[1] in top_two and percentage_by_age[2] in top_two
            if is_2030_in_top_two:
                self.store_dict["age-2030"] = True
            else:
                self.store_dict["age-2030"] = False

            # 남성의 비율이 50%를 넘는지 확인
            gender_xpath = """//*[@id="pie_chart_container"]/div/*[local-name()='svg']/*[local-name()='g'][1]/*[local-name()='g'][3]/*[local-name()='g'][4]/*[local-name()='g']/*[local-name()='text'][2]"""
            gender_elements = self.wait.until(
                EC.presence_of_all_elements_located((By.XPATH, gender_xpath)))
            female, male = [round(float(item.text.replace("%", "")), 0)
                            for item in gender_elements]
            if male > 50:
                self.store_dict["gender-balance"] = False
            else:
                self.store_dict["gender-balance"] = True
        except NoSuchElementException:
            self.store_dict["age-2030"] = None
            self.store_dict["gender-balance"] = None
        except Exception as e:
            print("데이터랩 에러: ", e)
            self.store_dict["age-2030"] = None
            self.store_dict["gender-balance"] = None


        # 블루 리본 등재 여부 -> "맛집" 키워드일 경우에만 확인
        if self.store_dict["name"].replace(" ", "") in [name.replace(" ", "") for name in self.blue_ribbon["name"].values]:
            indices = self.blue_ribbon.index[self.blue_ribbon["name"]
                                             == self.store_dict["name"].replace(" ", "")].tolist()

            for i, index in enumerate(indices):
                # 1. 도로명 주소 비교
                # - 띄어쓰기 모두 제거한 상태로 비교
                address1 = self.store_dict["address"].replace(" ", "")
                address2 = self.blue_ribbon["address"][index].replace(
                    " ", "")

                if address1 == address2:
                    print("도로명 주소 일치!!")
                    self.store_dict["on_blue_ribbon"] = True
                    break
                # 2. 위도-경도 비교
                lat1 = float(self.store_dict["gps"]["latitude"])
                lon1 = float(self.store_dict["gps"]["longitude"])

                lat2 = float(self.blue_ribbon["latitude"][index])
                lon2 = float(self.blue_ribbon["longitude"][index])
                distance = haversine(lat1, lon1, lat2, lon2)
                if distance <= 50:
                    print("위도-경도 근사!!")
                    self.store_dict["on_blue_ribbon"] = True
                    break
                if i + 1 == len(indices):
                    print("이름만 같았던 매장")
                    self.store_dict["on_blue_ribbon"] = False
                    break
        elif self.store_dict["phone"] in [phone for phone in self.blue_ribbon["phone"].values]:
            self.store_dict["on_blue_ribbon"] = True
        else:
            self.store_dict["on_blue_ribbon"] = False

        # 인스타그램 크롤링(게시글 수, 팔로워 수)
        if self.store_dict['instagram_link'] != None:  # 인스타그램 계정이 있는 경우에만 실행
            try:
                instagram_embed_url = self.store_dict['instagram_link'] + "/embed"

                # 인스타그랩 탭으로 이동
                self.driver.switch_to.window(self.driver.window_handles[1])
                self.driver.get(instagram_embed_url)

                xpath = """//span[contains(., '팔로워') and contains(., '게시물')]/span/span"""
                elements = self.wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, xpath)))

                follower = convert_str_to_number(elements[0].text)
                post = convert_str_to_number(elements[1].text)
                self.store_dict["instagram_follower"] = follower
                self.store_dict["instagram_post"] = post
            except (NoSuchElementException, TimeoutException):
                self.store_dict['instagram_link'] = None
                self.store_dict["instagram_follower"] = None
                self.store_dict["instagram_post"] = None

            # 네이버지도 탭으로 복귀
            self.driver.switch_to.window(self.driver.window_handles[0])

        # 한 매장에 대한 크롤링 결과
        print(self.store_dict)

        new_item = pd.DataFrame([self.store_dict])
        self.data_clear = pd.concat([self.data_clear, new_item], ignore_index=True)

    # 한 페이지 크롤링
    def crawling_one_page(self):
        # 매장 모두 로딩
        if self.move_to_search_iframe() == False:  # "조건에 맞는 업체가 없습니다"
            return False
        store_count = self.scroll_to_end()

        # 조건에 해당되는 매장이 없는 경우
        if store_count == 0:
            return False

        # 세 번째 이후부터는 매칭되는 매장이 아닐 확률이 매우 높음
        if store_count > 3:
            store_count = 3

        # 매장을 돌면서 매칭되는 게 있는지 확인하고
        # 매칭되는 게 있으면 거기서 break
        # 만약 매칭되는 게 없이 마지막 index가 됐다 -> 매칭되는 게 하나도 없다고 인식하고 data_clear에 집어넣기
        for i in range(1, store_count + 1):
            print(f"==== {i} 번째 매장 확인 ====")

            # 좌표가 다를 경우
            if self.get_into_store(index=i) == False:
                continue
            # 좌표가 같을 경우, 해당 매장으로 진입
            self.get_store_details()
            return True

        # 매칭되는 매장 없음
        return False

    # 위도-경도 값을 기반으로 같은 매장인지 판별 (T/F)
    def check_same_store(self):
        distance = haversine(float(self.current_store["latitude"]), float(self.current_store["longitude"]), float(
            self.store_dict["gps"]["latitude"]), float(self.store_dict["gps"]["longitude"]))
        if distance > 50:
            return False
        else:
            return True

    # 한 페이지에 대한 매장 개수 반환
    def scroll_to_end(self):
        try:
            li_xpath = """//*[@id="_pcmap_list_scroll_container"]/ul/li"""
            store_elements = self.wait_short.until(
                EC.presence_of_all_elements_located((By.XPATH, li_xpath)))
            store_count = len(store_elements)
            self.driver.execute_script(
                "arguments[0].scrollIntoView(true);", store_elements[-1])

            while True:
                time.sleep(0.5)
                store_elements = self.wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, li_xpath)))
                new_store_count = len(store_elements)

                if store_count == new_store_count:
                    break
                store_count = new_store_count
                self.driver.execute_script(
                    "arguments[0].scrollIntoView(true);", store_elements[-1])
            return store_count

        except (NoSuchElementException, TimeoutException):
            print(f"{self.current_store["name"]}이라는 업체는 검색되지 않습니다.")
            # 정보를 찾을 수 없는 매장 따로 저장
            return 0

    # searchIframe으로 이동
    def move_to_search_iframe(self):
        self.move_to_default_content()
        self.wait.until(EC.frame_to_be_available_and_switch_to_it(
            (By.XPATH, self.search_iframe)))
        try:
            self.wait_short.until(EC.presence_of_element_located(
                (By.XPATH, self.empty_searchIframe)))
            return True
        except (NoSuchElementException, TimeoutException):
            return False  # "조건에 맞는 업체가 없습니다"

    def move_to_default_content(self):
        self.driver.switch_to.default_content()
        self.wait.until(EC.presence_of_element_located(
            (By.XPATH, self.empty_root)))

    def crawling(self):

        total_count = len(self.datapop_stores)

        for index, store in self.datapop_stores.iterrows():

            print("="*10 + f" {index+1}/{total_count} || {store["name"]} " + "="*10)

            self.current_store = store
            self.init_dictionary()
            self.search_keyword(self.current_store["name"])

            try:
                self.wait_short.until(EC.presence_of_element_located(
                    (By.XPATH, self.entry_iframe)))
                iframe_element = self.driver.find_element(
                    By.XPATH, self.entry_iframe)
                # entry Iframe으로 실제로 이동하기 전에 해당 매장에 대한 url 얻기
                iframe_src = iframe_element.get_attribute('src')

                # 매장의 위도-경도 / 고유 ID 얻기
                parsed_url = urlparse(iframe_src)
                query_params = parse_qs(parsed_url.query)
                latitude = float(query_params.get('y')[0])
                longitude = float(query_params.get('x')[0])

                # store_point = Point(longitude, latitude)
                self.store_dict["gps"] = {
                    "latitude": latitude,
                    "longitude": longitude
                }

                path_segments = parsed_url.path.split('/')
                store_id = path_segments[2]
                self.store_dict["store_id"] = store_id

                # 매장 정보가 매칭되지 않음
                # 다음 매장으로 넘어감
                if self.check_same_store() == False:
                    continue

                self.driver.switch_to.frame(iframe_element)
                self.driver.find_element(By.XPATH, self.empty_entryIframe)

                # 한 매장에 대한 정보 크롤링 및 저장
                if self.get_store_details() == True:
                    new_data = pd.DataFrame([self.current_store])
                    self.data_clear = pd.concat(
                        [self.data_clear, new_data], ignore_index=True)

            except TimeoutException:
                # 매장 목록 전부 확인
                # 조건에 맞는 매장을 찾음
                if self.crawling_one_page() == True:
                    new_data = pd.DataFrame([self.store_dict])
                    self.data_clear = pd.concat(
                        [self.data_clear, new_data], ignore_index=True)
                # 매장 목록을 다 찾아봐도 조건에 맞는 매장을 찾을 수 없는 경우
                else:
                    pass  # 그냥 다른 매장으로 넘어감. 따로 처리 X

        self.driver.quit()

In [7]:
if __name__ == "__main__":

    file_path = "data/datepop/shop_active.csv"
    columns_names = ["name", "address", "latitude", "longitude", "active"]

    datepop = pd.read_csv(file_path, header=None, names=columns_names)
    datepop_seoul = datepop[datepop["address"].str.contains("서울특별시")]
    datepop_seoul = datepop_seoul[(datepop_seoul["latitude"] != 0) & (
        datepop_seoul["longitude"] != 0)]

    seoul_hotspots = load_hotspots('seoul_hotspots.csv')

    bluer_data1 = pd.read_csv("data/blueribbon/서울 강남_bluer.csv", header=0)
    bluer_data2 = pd.read_csv("data/blueribbon/서울 강북_bluer.csv", header=0)
    bluer_data = pd.concat([bluer_data1, bluer_data2], ignore_index=True)

    crawler = DatePopCrawler(
        hotspots=seoul_hotspots, blue_ribbon=bluer_data, datepop_stores=datepop_seoul)
    crawler.crawling()

    crawler_data_unique = crawler.data_clear.drop_duplicates(
        subset='store_id', keep='first')
    crawler_data_unique.to_csv("data/datepop/shop_seoul_crawled.csv", index=True, encoding='utf-8-sig')

    crawler.driver.quit()

========== 1/366 || 동백꽃공방 ==========
==== 1 번째 매장 확인 ====
433
{'store_id': '1560103206', 'name': '동백꽃공방', 'category': '공방', 'is_food': True, 'new_store': False, 'instagram_link': 'http://www.instagram.com/camellia_sss', 'instagram_post': 255, 'instagram_follower': 3466, 'hot_spot': True, 'visitor_review_count': None, 'blog_review_count': 48, 'distance_from_subway': 433, 'on_tv': False, 'parking_available': False, 'no_kids': False, 'pet_available': False, 'seoul_michelin': False, 'age-2030': None, 'gender-balance': None, 'on_blue_ribbon': False, 'image_urls': [], 'address': '서울 종로구 동숭1길 12 B1', 'phone': '010-8009-0601', 'gps': {'latitude': 37.5799126, 'longitude': 127.0050488}}
========== 2/366 || 비트포비아 대학로점 ==========
106
{'store_id': '37663526', 'name': '비트포비아 대학로', 'category': '방탈출카페', 'is_food': True, 'new_store': False, 'instagram_link': 'https://www.instagram.com/beat_phobia', 'instagram_post': 829, 'instagram_follower': 4904, 'hot_spot': True, 'visitor_review_count': 74, 'blog_re

### 음식 or 놀거리로 데이터 분리

In [ ]:
food = ["이탈리아음식", "한식", "피자", "양식", "베트남음식", "돈가스", "멕시코,남미음식", "햄버거", "육류,고기요리", "소고기구이", "일식당", "돼지고기구이", "브런치", "일본식라면", "아이스크림",
        "초밥,롤", "과일", "마사지, 지압", "아시아음식", "태국음식", "중식당", "백반,가정식", "퓨전음식", "닭발", "양꼬치", "닭갈비", "곱창,막창,양", "다이어트,샐러드", "국수", "라면", "일식튀김,꼬치", "종합분식"]

alcohol = ["술집", "이자카야", "바(BAR)", "맥주,호프", "주류", "요리주점", "와인", "포장마차"]

cafe = ["카페", "카페,디저트", "와플", "도넛", "베이커리"]

playing = ["공방", "방탈출카페", "힐링카페", "사격장", "테마카페", "테마파크", "아쿠아리움",  "당구장", "주류제조", "꽃집,꽃배달", "볼링장", "애견카페", "사진,스튜디오",
           "고양이카페", "오락실", "낚시터", "스포츠,오락", "셀프,대여스튜디오", "찜질방", "서바이벌게임", "만화방", "갤러리,화랑", "유리공예", "공연,연극시설", "운세,사주", "한복대여", "경영컨설팅"]

place_lend = ["장소대여", "임대,대여"]

care = ["힐링카페", "발관리", "치과", "미용"]

movie = ["영화관", "DVD방"]

else_list = ["광고,마케팅", "화장품,향수", "피부,체형관리", "속옷", "공연장"]

In [ ]:
df = pd.read_csv('data/datepop/shop_seoul_crawled.csv', header=0, encoding='cp949')

df.drop('Unnamed: 0', axis=1, inplace=True)
print(len(df))

In [ ]:
# food 또는 alcohol 리스트에 해당하는 category를 가진 행들을 필터링
food_df = df[df['category'].isin(food + alcohol + cafe)].reset_index(drop=True)

# # food 또는 alcohol 리스트에 해당하지 않는 category를 가진 행들을 필터링
plyaing_df = df[df['category'].isin(playing + place_lend + care + movie + else_list)].reset_index(drop=True)

In [ ]:
food_df.to_csv('data/datepop/shop_food.csv', encoding='utf-8-sig')
plyaing_df.to_csv('data/datepop/shop_playing.csv', encoding='utf-8-sig')

In [ ]:
len(food_df) + len(plyaing_df)

In [ ]:
food_df.head(10)